In [2]:
import sqlite3
import csv

# Chemin du fichier CSV
csv_file_path = "raw_data/sales.csv"

# Connexion à la base de données SQLite
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

# Création de la table avec des types adaptés
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,           -- Format YYYY-MM-DD
    datetime TEXT,       -- Format YYYY-MM-DD HH:MM:SS.sss
    cash_type TEXT,
    card TEXT,
    money REAL,
    coffee_name TEXT
)
''')

# Lecture et insertion des données du fichier CSV
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    # Ignorer les en-têtes
    headers = next(reader)
    
    # Insérer les lignes dans la table SQLite
    for row in reader:
        cursor.execute('''
        INSERT INTO sales (date, datetime, cash_type, card, money, coffee_name)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', row)

# Sauvegarder les changements et fermer la connexion
conn.commit()
conn.close()

print("Les données ont été importées dans la base de données 'sales.db'.")

Les données ont été importées dans la base de données 'sales.db'.


In [3]:
import sqlite3
import pandas as pd

# Connexion à la base de données SQLite
conn = sqlite3.connect("sales.db")

# Charger les données de la table 'sales' dans un DataFrame Pandas
query = "SELECT * FROM sales"
data = pd.read_sql_query(query, conn)

# Conversion des colonnes 'date' et 'datetime' en type datetime
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S.%f')

# Afficher des informations générales sur les données
print("Informations générales sur les données :")
print(data.info())

# Afficher les 5 premières lignes des données
print("\nPremières lignes des données :")
print(data.head())

# Fermeture de la connexion
conn.close()

Informations générales sur les données :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13115 entries, 0 to 13114
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           13115 non-null  int64         
 1   date         13115 non-null  datetime64[ns]
 2   datetime     13115 non-null  datetime64[ns]
 3   cash_type    13115 non-null  object        
 4   card         13115 non-null  object        
 5   money        13115 non-null  float64       
 6   coffee_name  13115 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 717.4+ KB
None

Premières lignes des données :
   id       date                datetime cash_type                 card  \
0   1 2024-03-01 2024-03-01 10:15:50.520      card  ANON-0000-0000-0001   
1   2 2024-03-01 2024-03-01 12:19:22.539      card  ANON-0000-0000-0002   
2   3 2024-03-01 2024-03-01 12:20:18.089      card  ANON-0000-000

In [4]:
import sqlite3
import pandas as pd
from datetime import datetime

# Connexion à la base de données SQLite
conn = sqlite3.connect("sales.db")

# Charger les données de la table 'sales' dans un DataFrame Pandas
query = "SELECT * FROM sales"
data = pd.read_sql_query(query, conn)

# Conversion des colonnes 'date' et 'datetime' en type datetime
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S.%f')

# Trouver la dernière date dans les données
derniere_date = data['date'].max()
dernier_mois = derniere_date.month
derniere_annee = derniere_date.year

# Déterminer le mois et l'année du mois précédent
if dernier_mois == 1:  # Si janvier, alors le mois précédent est décembre de l'année précédente
    mois_precedent = 12
    annee_precedente = derniere_annee - 1
else:
    mois_precedent = dernier_mois - 1
    annee_precedente = derniere_annee

# Filtrer les données pour le dernier mois
data_dernier_mois = data[(data['date'].dt.month == dernier_mois) & (data['date'].dt.year == derniere_annee)]

# Filtrer les données pour le mois précédent
data_mois_precedent = data[(data['date'].dt.month == mois_precedent) & (data['date'].dt.year == annee_precedente)]

# Calcul des KPIs pour le dernier mois
total_ca_dernier_mois = round(data_dernier_mois['money'].sum(), 0)
nombre_transactions_dernier_mois = len(data_dernier_mois)
moyenne_ca_dernier_mois = round(data_dernier_mois['money'].mean(), 0)
produit_populaire_dernier_mois = data_dernier_mois['coffee_name'].mode()[0]

# Calcul des KPIs pour le mois précédent
total_ca_mois_precedent = round(data_mois_precedent['money'].sum(), 0)
nombre_transactions_mois_precedent = len(data_mois_precedent)
moyenne_ca_mois_precedent = round(data_mois_precedent['money'].mean(), 0)
produit_populaire_mois_precedent = data_mois_precedent['coffee_name'].mode()[0]

# Calcul des pourcentages de variation
pourcentage_variation_ca = ((total_ca_dernier_mois - total_ca_mois_precedent) / total_ca_mois_precedent) * 100 if total_ca_mois_precedent != 0 else 0
pourcentage_variation_transactions = ((nombre_transactions_dernier_mois - nombre_transactions_mois_precedent) / nombre_transactions_mois_precedent) * 100 if nombre_transactions_mois_precedent != 0 else 0
pourcentage_variation_moyenne_ca = ((moyenne_ca_dernier_mois - moyenne_ca_mois_precedent) / moyenne_ca_mois_precedent) * 100 if moyenne_ca_mois_precedent != 0 else 0

# Stocker les résultats dans une variable texte
resultats_kpis = f"""
KPIs pour le dernier mois :
- Total du chiffre d'affaires : {total_ca_dernier_mois}
- Nombre de transactions : {nombre_transactions_dernier_mois}
- Moyenne du chiffre d'affaires par transaction : {moyenne_ca_dernier_mois}
- Produit le plus populaire : {produit_populaire_dernier_mois}

KPIs pour le mois précédent :
- Total du chiffre d'affaires : {total_ca_mois_precedent}
- Nombre de transactions : {nombre_transactions_mois_precedent}
- Moyenne du chiffre d'affaires par transaction : {moyenne_ca_mois_precedent}
- Produit le plus populaire : {produit_populaire_mois_precedent}

Tendances en pourcentage par rapport au mois précédent :
- Variation du chiffre d'affaires : {pourcentage_variation_ca:.2f}%
- Variation des transactions : {pourcentage_variation_transactions:.2f}%
- Variation de la moyenne du chiffre d'affaires : {pourcentage_variation_moyenne_ca:.2f}%
"""

# Afficher les résultats
print(resultats_kpis)

# Fermeture de la connexion
conn.close()



KPIs pour le dernier mois :
- Total du chiffre d'affaires : 30265.0
- Nombre de transactions : 945
- Moyenne du chiffre d'affaires par transaction : 32.0
- Produit le plus populaire : Americano with Milk

KPIs pour le mois précédent :
- Total du chiffre d'affaires : 42953.0
- Nombre de transactions : 1295
- Moyenne du chiffre d'affaires par transaction : 33.0
- Produit le plus populaire : Latte

Tendances en pourcentage par rapport au mois précédent :
- Variation du chiffre d'affaires : -29.54%
- Variation des transactions : -27.03%
- Variation de la moyenne du chiffre d'affaires : -3.03%

